In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [3]:
# Define cubic SE1, find and classify the lines and compute the 45 tritangent planes

In [4]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [5]:
# Find simmetries

In [6]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 1.43 s, sys: 121 ms, total: 1.55 s
Wall time: 10.5 s


576

In [7]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 1.73 s, sys: 96.8 ms, total: 1.82 s
Wall time: 10.7 s


2

In [8]:
# Study how the simmetries permute the Eckardt points

In [9]:
Eck_points_perms = []
for proj in simm_SE1:
    Eck_points_perms.append(Permutation(SE1.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(1, '1')

In [10]:
# Study how the simmetries permute the 27 lines

In [11]:
lines_perms = []
for simm in simm_SE1:
    lines_perms.append(Permutation(SE1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(2, 'C2')

In [12]:
lines_perms_group.gens_small()

[(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23)]

In [13]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('E1', 'G4', 'F24', 'F13', 'F34', 'F45')]

In [20]:
all_perms = find_all_permutations()

In [22]:
all_permutation = [from_labels_to_perm(label) for label in all_perms]

In [23]:
order2_perms = [perm for perm in all_permutation if perm.order() == 2]

In [24]:
len(order2_perms)

891

In [33]:
possible_L_sets = list(set([get_permuted_L_set(perm) for perm in order2_perms]))

In [34]:
mon = ((x+y+z+t)^3).monomials()
L_set_base = ['E1', 'G4', 'E2', 'G3', 'E3']

In [35]:
%time possible_projs = [find_projectivity(SE1.get_L_set_in_plucker(L_set_base), SE1.get_L_set_in_plucker(L_set)) for L_set in possible_L_sets]

CPU times: user 2min 17s, sys: 399 ms, total: 2min 17s
Wall time: 2min 17s


In [36]:
len(possible_projs)

816

In [37]:
SE1.find_simmetries(possible_projs)

[
[                -2*c^2                      0                      0 -c^2 + c*e + c*f - e*f]
[                     0                 -2*c^2                      0        c*d - c*e - c*f]
[                     0                      0                 -2*c^2        c^2 - c*d + e*f]
[                     0                      0                      0                  2*c^2],

[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]
]

In [19]:
simm_SE1

[
[1 0 0 0]  [                -2*c^2                      0                      0 -c^2 + c*e + c*f - e*f]
[0 1 0 0]  [                     0                 -2*c^2                      0        c*d - c*e - c*f]
[0 0 1 0]  [                     0                      0                 -2*c^2        c^2 - c*d + e*f]
[0 0 0 1], [                     0                      0                      0                  2*c^2]
]

In [39]:
possible_L_sets = [('E1', 'G4', 'E2', 'F12', 'E3'),
                   ('E1', 'G4', 'E2', 'G5', 'F34'),
                   ('E1', 'G4', 'E2', 'G6', 'F34'),
                   ('E1', 'G4', 'F24', 'G2', 'F34'),
                   ('E1', 'G4', 'F24', 'F13', 'F34'),
                   ('F14', 'G4', 'F24', 'F35', 'F34'),
                   ('F14', 'G4', 'F24', 'F36', 'F34'),
                   ('E1', 'G4', 'E3', 'G2', 'E2'),
                   ('E1', 'G4', 'E3', 'F13', 'E2'),
                   ('F14', 'G4', 'E3', 'F35', 'E2'),
                   ('F14', 'G4', 'E3', 'F36', 'E2')] 

In [40]:
possible_projs = [find_projectivity(SE1.get_L_set_in_plucker(L_set_base), SE1.get_L_set_in_plucker(L_set)) for L_set in possible_L_sets]

In [41]:
SE1.find_conditions_for_subfamilies(possible_projs, simm_SE1)

[]